# Setup and Imports

In [1]:
import os
from pathlib import Path

import numpy as np
import polars as pl
from dotenv import load_dotenv
from numba import config

from src.tann.evolution.individual import predict_individual, initialize_individual, individual_actions
from src.tann.evolution.population import initialize_population, get_population_actions
from src.tann.network.neuron import compute_layer_indices

# Auto-reload modules
%load_ext autoreload
%autoreload 2

# Load environment and set working directory
load_dotenv()
root = Path(os.getenv("ROOT"))
os.chdir(root)

# Configuration

In [2]:
# Objective parameters
n_features = 1
output_size = 3
output_activation = 2
features = ["price"]
timestamp_col = "timestamp"

# Individual parameters
hidden_layers_sizes = [10, 10]
hidden_layers_activations = [1, 1, 1]

# Helpers
seed = 123
epoch_id = 0

# Numba configuration

# Load and Prepare Data

In [3]:
df = pl.read_parquet(root / 'data/mock.parquet')
df = df.with_columns(
    (pl.col('timestamp') // 600_000).alias('epoch'),
)

# Initialize Network Architecture

In [4]:
layer_sizes = [n_features] + hidden_layers_sizes + [output_size]
layer_activations = hidden_layers_activations + [output_activation]
previous_states = np.zeros(np.sum(layer_sizes[1:]), dtype=np.float64)
previous_time = df.filter(pl.col('epoch') == epoch_id).select('timestamp').to_series()[0]

# Compute layer indices for efficient processing
param_indices, neuron_indices = compute_layer_indices(layer_sizes)

# Initialize Individual

In [5]:
individual = initialize_individual(layer_sizes, seed=seed)

# Prepare Data for Testing

In [6]:
df_epoch = df.filter(pl.col("epoch") == epoch_id)

# Extract timestamps and feature values
timestamps = df_epoch[timestamp_col].to_numpy()
feature_values = df_epoch.select(features).to_numpy()

# Prepare single input for testing
current_time = df_epoch.select(timestamp_col).to_series()[0]
values = np.array([df_epoch.select(features).to_series()[0]], dtype=np.float64)
inputs = (current_time, values)

# Test Individual Prediction

In [7]:
current_values, new_states = predict_individual(
    individual, layer_sizes, layer_activations, inputs, 
    previous_states, previous_time, param_indices, neuron_indices
)

# Benchmark Individual Prediction

In [8]:
%timeit -r 1 current_values, new_states = predict_individual(individual, layer_sizes, layer_activations, inputs, previous_states, previous_time, param_indices, neuron_indices)

5.22 μs ± 0 ns per loop (mean ± std. dev. of 1 run, 100,000 loops each)


# Benchmark Individual Actions

In [9]:
%timeit -r 1 actions = individual_actions(individual, layer_sizes, layer_activations, previous_states, previous_time, param_indices, neuron_indices, timestamps, feature_values)

2.11 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 100 loops each)


# Initialize Population

In [10]:
# Initialize population
pop_size = 100
population = initialize_population(pop_size, layer_sizes, seed=seed)

# Initialize population states
population_states = np.zeros((pop_size, np.sum(layer_sizes[1:])), dtype=np.float64)

# Benchmark Population Actions

In [11]:
%timeit -r 1 get_population_actions_result = get_population_actions(population, layer_sizes, layer_activations, population_states, previous_time, param_indices, neuron_indices, timestamps, feature_values)

98.1 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 10 loops each)
